This code loads the CSVs from your drive and calculates the instantaneous power output and saves the update CSVs for each city.

If you want to run this on your own, change the file path in the fifth line to the drive folder where you store the files.


Maya

In [1]:
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')
#change this path to the folder where you store the city files to run on your own device!
drive_path = '/content/drive/My Drive/city-files/'

cities = ["Seattle", "NYC", "LA", "Miami", "LV", "Denver"]
csv_files = [f"{city}.csv" for city in cities]

# specs from the solar panel manual
# assumes CS6R of 400 MS
Pmax = 400
MEff = 0.205
TCff = -0.0034
NOCT = 42
Area = 1.953  # in meters sq
Sys = 0.9

for city, filename in zip(cities, csv_files):
    path = os.path.join(drive_path, filename)
    try:
      df = pd.read_csv(path, skiprows=2,header=0)
      df.columns = df.columns.str.strip()

      df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
      df = df.drop(columns=[col for col in df.columns if 'default_value' in col], errors='ignore')


      print(f"\ncolumns for {city}:")
      print(df.columns.tolist())
      df = df.dropna(subset=["GHI",'Temperature'])

      df['GHI '] = pd.to_numeric(df['GHI'], errors='coerce')
      df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')

      #calculate the values
      df['CTemp'] = df['Temperature'] + (df['GHI'] / 800) * (NOCT - 20)
      df['PowerOutput_W'] = Pmax * (df['GHI'] / 1000) * (1 + TCff * (df['CTemp'] - 25)) * Sys

      # save updated file
      out_path = os.path.join(drive_path, f"{city}_with_power.csv")
      df.to_csv(out_path, index=False)
      print(f"saved: {out_path}")
    except Exception as e:
        print(f"error processing {city}: {e}")

Mounted at /content/drive
error processing Seattle: [Errno 2] No such file or directory: '/content/drive/My Drive/city-files/Seattle.csv'
error processing NYC: [Errno 2] No such file or directory: '/content/drive/My Drive/city-files/NYC.csv'
error processing LA: [Errno 2] No such file or directory: '/content/drive/My Drive/city-files/LA.csv'
error processing Miami: [Errno 2] No such file or directory: '/content/drive/My Drive/city-files/Miami.csv'
error processing LV: [Errno 2] No such file or directory: '/content/drive/My Drive/city-files/LV.csv'
error processing Denver: [Errno 2] No such file or directory: '/content/drive/My Drive/city-files/Denver.csv'
